In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-29 14:45:31.568421: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-29 14:45:31.592807: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-29 14:45:31.592832: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-29 14:45:31.592856: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-29 14:45:31.597896: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-29 14:45:33.324576: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 14:45:33.347206: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 14:45:33.349090: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [3]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [4]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [5]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [6]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.


2024-07-29 14:45:33.430953: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 14:45:33.433753: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-29 14:45:33.435733: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [7]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [8]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.resnet_rs.preprocess_input

In [9]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [10]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [11]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.ResNetRS101(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [12]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: rescaling, False
Layer: normalization, False
Layer: zero_padding2d, False
Layer: stem_1_stem_conv_1, False
Layer: stem_1_stem_batch_norm_1, False
Layer: stem_1_stem_act_1, False
Layer: stem_1_stem_conv_2, False
Layer: stem_1_stem_batch_norm_2, False
Layer: stem_1_stem_act_2, False
Layer: stem_1_stem_conv_3, False
Layer: stem_1_stem_batch_norm_3, False
Layer: stem_1_stem_act_3, False
Layer: zero_padding2d_1, False
Layer: stem_1_stem_conv_4, False
Layer: stem_1_stem_batch_norm_4, False
Layer: stem_1_stem_act_4, False
Layer: BlockGroup2__block_0__conv_1, False
Layer: BlockGroup2__block_0_batch_norm_1, False
Layer: BlockGroup2__block_0__act_1, False
Layer: BlockGroup2__block_0__conv_2, False
Layer: BlockGroup2__block_0__batch_norm_2, False
Layer: BlockGroup2__block_0__act_2, False
Layer: BlockGroup2__block_0__conv_3, False
Layer: BlockGroup2__block_0__batch_norm_3, False
Layer: BlockGroup2__block_0__se_se_squeeze, False
Layer: BlockGroup2__block_0__se_se_reshap

In [13]:
len(base_model.layers)

526

In [14]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [15]:
aug_layer = utils.return_data_aug_layer_for_eff_net()

In [16]:
from keras.src.regularizers import l2
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout, BatchNormalization
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
x = aug_layer(inputs)
#x = tf.keras.layers.Rescaling(1./255)(inputs)
x = base_model(x, training = False)
x = layers.GlobalAvgPool2D()(x)
x = BatchNormalization()(x)

x = Dense(512, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)

x = Dense(256, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)

x = Dense(128, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)


x = Dense(64, kernel_regularizer=l2(0.01))(x)
x = BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = Dropout(0.3)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'Resnet_rs_101')

model_1.summary()

Model: "Resnet_rs_101"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet-rs-101 (Functional)  (None, 7, 7, 2048)        61675296  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                     

In [17]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [18]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=3) 

In [19]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [20]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 20, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/20


2024-07-29 14:45:49.448670: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-29 14:45:49.496274: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-07-29 14:45:51.064587: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x758ff8003870 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-07-29 14:45:51.064615: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-07-29 14:45:51.070287: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-29 14:45:51.128576: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


74/74 [==============================] - ETA: 0s - loss: 13.0940 - accuracy: 0.6099
Epoch 1: val_loss improved from inf to 10.46431, saving model to Trained_Models/Resnet_rs_101
74/74 [==============================] - 63s 762ms/step - loss: 13.0940 - accuracy: 0.6099 - val_loss: 10.4643 - val_accuracy: 0.6973 - lr: 0.0010
Epoch 2/20
74/74 [==============================] - ETA: 0s - loss: 7.9376 - accuracy: 0.7522
Epoch 2: val_loss improved from 10.46431 to 5.93901, saving model to Trained_Models/Resnet_rs_101
74/74 [==============================] - 47s 630ms/step - loss: 7.9376 - accuracy: 0.7522 - val_loss: 5.9390 - val_accuracy: 0.9456 - lr: 0.0010
Epoch 3/20
74/74 [==============================] - ETA: 0s - loss: 4.7237 - accuracy: 0.7738
Epoch 3: val_loss improved from 5.93901 to 3.81777, saving model to Trained_Models/Resnet_rs_101
74/74 [==============================] - 47s 636ms/step - loss: 4.7237 - accuracy: 0.7738 - val_loss: 3.8178 - val_accuracy: 0.8259 - lr: 0.0010
Ep

In [21]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 2s 78ms/step - loss: 0.8670 - accuracy: 0.7787


[0.8670157790184021, 0.778698205947876]

In [22]:
base_model.trainable = True

In [23]:
for layer in base_model.layers[:-25]:
    layer.trainable = False

In [24]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [25]:
start_epoch = 10
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet_rs_101"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 resnet-rs-101 (Functional)  (None, 7, 7, 2048)        61675296  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                                 
 dense (Dense)               (None, 512)               1049088   
                                                                 
 batch_normalization_1 (Bat  (None, 512)             

In [26]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 11/20
74/74 [==============================] - ETA: 0s - loss: 0.8134 - accuracy: 0.8513
Epoch 11: val_loss improved from 0.69015 to 0.59161, saving model to Trained_Models/Resnet_rs_101
74/74 [==============================] - 33s 374ms/step - loss: 0.8134 - accuracy: 0.8513 - val_loss: 0.5916 - val_accuracy: 0.8991 - lr: 0.0010
Epoch 12/20
74/74 [==============================] - ETA: 0s - loss: 0.6325 - accuracy: 0.8645
Epoch 12: val_loss did not improve from 0.59161
74/74 [==============================] - 9s 119ms/step - loss: 0.6325 - accuracy: 0.8645 - val_loss: 1.8722 - val_accuracy: 0.2740 - lr: 0.0010
Epoch 13/20
74/74 [==============================] - ETA: 0s - loss: 0.5890 - accuracy: 0.8755
Epoch 13: val_loss did not improve from 0.59161
74/74 [==============================] - 9s 119ms/step - loss: 0.5890 - accuracy: 0.8755 - val_loss: 0.7865 - val_accuracy: 0.8229 - lr: 0.0010
Epoch 14/20
74/74 [==============================] - ETA: 0s - loss: 0.5366 - accuracy: 

In [22]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 2s 63ms/step - loss: 0.8670 - accuracy: 0.7787


[0.8670157790184021, 0.778698205947876]

In [23]:
start_epoch = 20
for layer in base_model.layers[-25:]:
    layer.trainable = True

In [24]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet_rs_101"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet-rs-101 (Functional)  (None, 7, 7, 2048)        61675296  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                     

In [25]:
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 21/30
74/74 [==============================] - ETA: 0s - loss: 0.6629 - accuracy: 0.8488
Epoch 21: val_loss did not improve from 0.55286
74/74 [==============================] - 21s 217ms/step - loss: 0.6629 - accuracy: 0.8488 - val_loss: 0.5706 - val_accuracy: 0.9041 - lr: 3.0000e-04
Epoch 22/30
74/74 [==============================] - ETA: 0s - loss: 0.6436 - accuracy: 0.8590
Epoch 22: val_loss improved from 0.55286 to 0.46664, saving model to Trained_Models/Resnet_rs_101
74/74 [==============================] - 36s 480ms/step - loss: 0.6436 - accuracy: 0.8590 - val_loss: 0.4666 - val_accuracy: 0.9476 - lr: 3.0000e-04
Epoch 23/30
74/74 [==============================] - ETA: 0s - loss: 0.6401 - accuracy: 0.8602
Epoch 23: val_loss did not improve from 0.46664
74/74 [==============================] - 14s 192ms/step - loss: 0.6401 - accuracy: 0.8602 - val_loss: 0.7798 - val_accuracy: 0.7933 - lr: 3.0000e-04
Epoch 24/30
74/74 [==============================] - ETA: 0s - loss: 0.649

In [26]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 2s 67ms/step - loss: 0.6451 - accuracy: 0.8331


[0.645103394985199, 0.8331360816955566]

In [35]:
start_epoch = 30
base_model.trainable = True
for layer in base_model.layers[:-25]:
    layer.trainable = False

In [36]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000075), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet_rs_101"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet-rs-101 (Functional)  (None, 7, 7, 2048)        61675296  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                     

In [37]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 31/40
74/74 [==============================] - ETA: 0s - loss: 0.4646 - accuracy: 0.8878
Epoch 31: val_loss did not improve from 0.28269
74/74 [==============================] - 23s 199ms/step - loss: 0.4646 - accuracy: 0.8878 - val_loss: 0.4277 - val_accuracy: 0.9050 - lr: 7.5000e-05
Epoch 32/40
74/74 [==============================] - ETA: 0s - loss: 0.4084 - accuracy: 0.9089
Epoch 32: val_loss did not improve from 0.28269
74/74 [==============================] - 12s 162ms/step - loss: 0.4084 - accuracy: 0.9089 - val_loss: 0.3359 - val_accuracy: 0.9466 - lr: 7.5000e-05
Epoch 33/40
74/74 [==============================] - ETA: 0s - loss: 0.4009 - accuracy: 0.9136
Epoch 33: val_loss improved from 0.28269 to 0.27764, saving model to Trained_Models/Resnet_rs_101
74/74 [==============================] - 33s 450ms/step - loss: 0.4009 - accuracy: 0.9136 - val_loss: 0.2776 - val_accuracy: 0.9634 - lr: 7.5000e-05
Epoch 34/40
74/74 [==============================] - ETA: 0s - loss: 0.424

In [38]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 2s 64ms/step - loss: 0.5906 - accuracy: 0.8627


[0.5906388759613037, 0.8627219200134277]

In [39]:
start_epoch = 40
for layer in base_model.layers[-50:]:
    layer.trainable = True

In [40]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.000025), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet_rs_101"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet-rs-101 (Functional)  (None, 7, 7, 2048)        61675296  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 batch_normalization (Batch  (None, 2048)              8192      
 Normalization)                                                  
                                                     

In [41]:
history_500 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+10, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 41/50
74/74 [==============================] - ETA: 0s - loss: 0.3422 - accuracy: 0.9276
Epoch 41: val_loss improved from 0.23144 to 0.20779, saving model to Trained_Models/Resnet_rs_101
74/74 [==============================] - 47s 536ms/step - loss: 0.3422 - accuracy: 0.9276 - val_loss: 0.2078 - val_accuracy: 0.9773 - lr: 2.5000e-05
Epoch 42/50
74/74 [==============================] - ETA: 0s - loss: 0.3325 - accuracy: 0.9348
Epoch 42: val_loss did not improve from 0.20779
74/74 [==============================] - 13s 176ms/step - loss: 0.3325 - accuracy: 0.9348 - val_loss: 0.2978 - val_accuracy: 0.9456 - lr: 2.5000e-05
Epoch 43/50
74/74 [==============================] - ETA: 0s - loss: 0.3403 - accuracy: 0.9238
Epoch 43: val_loss did not improve from 0.20779
74/74 [==============================] - 13s 173ms/step - loss: 0.3403 - accuracy: 0.9238 - val_loss: 0.2304 - val_accuracy: 0.9753 - lr: 2.5000e-05
Epoch 44/50
74/74 [==============================] - ETA: 0s - loss: 0.329

In [42]:
model_1.evaluate(test_datagen)

27/27 [==============================] - 2s 61ms/step - loss: 0.6092 - accuracy: 0.8568


[0.6092352271080017, 0.8568047285079956]

In [38]:
resnet = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet")


OSError: No file or directory found at /home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet

In [ ]:
resnet.evaluate(val_datagen)